In [2]:
import os
import json
import glob

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.model_selection import StratifiedKFold


In [3]:
import time
import psutil
from contextlib import contextmanager

def show_memory_usage(name = "unknown"):
    vm = psutil.virtual_memory()
    print(f"[MEMUSE] memory usage (in {name}): {vm.used/1024/1024:.2f}MB ({vm.percent}%)")

@contextmanager
def timer(name: str):
    show_memory_usage(f"before {name}")
    s = time.time()
    yield
    elapsed = time.time() - s
    print(f"[{name}] {elapsed:.3f}sec")
    show_memory_usage(f"after {name}")

In [4]:
class Config:
    def __init__(self):
        self.steps_per_sec = 0.2
        self.step_for_a_day = 60 * self.steps_per_sec * 60 * 24
        self.step_for_30min = 60 * self.steps_per_sec * 30
        self.step_for_15min = 60 * self.steps_per_sec * 15
        self.step_for_1min = 60 * self.steps_per_sec

    def from_json(self, json_path):
        json_data = json.load(open(json_path))
        for k, v in json_data.items():
            print(k, v)
            setattr(self, k, v)
        return self

setting_file = "SETTINGS.json"
Cfg = Config().from_json(setting_file)



In [5]:
%load_ext cython

In [6]:
%%cython
import numpy as np
cimport numpy as cnp
cimport cython

def cumsum_morethan_zero(cnp.ndarray[cnp.float64_t, ndim=1] x):
    cdef int i, n
    n = x.shape[0]
    cdef cnp.ndarray[cnp.float64_t, ndim=1] y = np.zeros(n)
    cdef cnp.ndarray[cnp.float64_t, ndim=1] y_rev = np.zeros(n)
    y[0] = x[0]
    for i in range(1, n):
        if x[i] == 0:
            y[i] = 0
        else:
            y[i] = y[i-1] + x[i]
    y_rev[-1] = y[-1]
    for i in range(n-2, -1, -1):
        if y_rev[i+1] > y[i]:
            if x[i] == 0:
                y_rev[i] = 0
            else:
                y_rev[i] = y_rev[i+1]
        else:
            y_rev[i] = y[i]
    return y_rev

def easy_convolve(cnp.ndarray[cnp.float64_t, ndim=1] x, int filter_size):
    """
    padding same, kernel is ones
    """
    cdef int i, j, n, p, m
    m = filter_size - 1
    p = m // 2
    n = x.shape[0]
    cdef cnp.ndarray[cnp.float64_t, ndim=1] x_p = np.zeros(n+2*p)
    cdef cnp.ndarray[cnp.float64_t, ndim=1] y = np.zeros(n)
    x_p[p:n+p] = x

    for j in range(filter_size):
        y[0] += x_p[j]

    for i in range(1, n):# filter_size, n+p+p-filter_size+1):
        y[i] = x_p[i+m] + y[i-1] - x_p[i-1]
    return y

def minimum(cnp.ndarray[cnp.float64_t, ndim=1] x, cnp.float64_t maxval):
    cdef int i, n
    n = x.shape[0]
    cdef cnp.ndarray[cnp.float64_t, ndim=1] y = np.zeros(n)
    for i in range(n):
        y[i] = min(x[i], maxval)
    return y

def easy_closing(cnp.ndarray[cnp.float64_t, ndim=1] x, int filter_size):
    """
    closing = dilation -> erosion
    padding same, kernel is ones, x is 0 or 1
    """
    x = easy_convolve(x, filter_size)
    x = minimum(x, 1)
    x = 1 - x
    x = easy_convolve(x, filter_size)
    x = minimum(x, 1)
    x = 1 - x
    return x

def easy_closing_q(cnp.ndarray[cnp.float64_t, ndim=1] x, int filter_size):
    """
    closing = dilation -> erosion
    padding same, kernel is ones, x is 0 or 1
    少し早いけどわかりにくい…。
    """
    cdef int i, j, n, p, m
    m = filter_size - 1
    p = m // 2
    n = x.shape[0]
    cdef cnp.ndarray[cnp.float64_t, ndim=1] x_p = np.zeros(n+2*p)
    cdef cnp.ndarray[cnp.float64_t, ndim=1] y_p = np.zeros(n+2*p)
    cdef cnp.ndarray[cnp.float64_t, ndim=1] y = np.zeros(n)
    cdef cnp.ndarray[cnp.float64_t, ndim=1] z = np.zeros(n)
    
    x_p[p:n+p] = x
    for j in range(filter_size):
        y[0] += x_p[j]
    for i in range(1, n):# filter_size, n+p+p-filter_size+1):
        y[i] = x_p[i+m] + y[i-1] - x_p[i-1]
    for i in range(n):
        y[i] = 1 - min(y[i], 1)
    
    y_p[p:n+p] = y
    for j in range(filter_size):
        z[0] += y_p[j]
    for i in range(1, n):# filter_size, n+p+p-filter_size+1):
        z[i] = y_p[i+m] + z[i-1] - y_p[i-1]
    for i in range(n):
        z[i] = 1 - min(z[i], 1)
    
    return z


def _detect_peak(cnp.ndarray[cnp.float64_t, ndim=1] x, int k):
    cdef int n = x.shape[0]
    cdef cnp.ndarray[cnp.float64_t, ndim=1] max_array = np.zeros(n, dtype=np.float64)
    cdef cnp.ndarray[cnp.int32_t, ndim=1] max_indices = np.zeros(n, dtype=np.int32)
    cdef cnp.ndarray[cnp.float64_t, ndim=1] result = np.zeros(n, dtype=np.float64)
    cdef int i, j, start, end, max_index
    
    # calculate max values in each window
    for i in range(n):
        start = max(0, i - k)
        end = min(n, i + k + 1)
        max_index = start
        for j in range(start, end):
            if x[j] > x[max_index]:
                max_index = j
        max_array[i] = x[max_index]
        max_indices[i] = max_index
    
    # set peak values to 1
    for i in range(n):
        if x[i] == max_array[max_indices[i]]:
            result[i] = 1.0
    
    return max_array

def _detect_peak_r(cnp.ndarray[cnp.float64_t, ndim=1] x, int k):
    cdef int n = x.shape[0]
    cdef cnp.ndarray[cnp.float64_t, ndim=1] max_array = np.zeros(n, dtype=np.float64)
    cdef cnp.ndarray[cnp.int32_t, ndim=1] max_indices = np.zeros(n, dtype=np.int32)
    cdef cnp.ndarray[cnp.float64_t, ndim=1] result = np.zeros(n, dtype=np.float64)
    cdef int i, j, start, end, max_index
    
    # calculate max values in first window
    max_index = 0
    for i in range(k):
        if x[i] > x[max_index]:
            max_index = i
    max_array[k-1] = x[max_index]
    max_indices[k-1] = max_index
    
    # calculate max values in each window
    for i in range(k, n):
        start = i - k
        end = i
        if max_index == start - 1:
            max_index = start
            for j in range(start, end):
                if x[j] > x[max_index]:
                    max_index = j
        else:
            if x[i] > x[max_index]:
                max_index = i
        max_array[i] = x[max_index]
        max_indices[i] = max_index
    
    # set peak values to 1
    for i in range(n):
        if x[i] == max_array[max_indices[i]]:
            result[i] = 1.0
    
    return max_array

@cython.boundscheck(False)
@cython.wraparound(False)
def detect_peak_kmat(cnp.ndarray[cnp.float64_t, ndim=1] x, int k):
    cdef int n = x.shape[0]
    cdef cnp.ndarray[cnp.float64_t, ndim=1] max_array = np.zeros(n, dtype=np.float64)
    cdef cnp.ndarray[cnp.float64_t, ndim=1] result_val = np.zeros(n, dtype=np.float64)
    cdef cnp.ndarray[cnp.float64_t, ndim=1] result = np.zeros(n, dtype=np.float64)
    cdef int i, j, start, end, max_index, half_k

    half_k = k // 2
    for i in range(half_k, n-half_k):
        result[i] = 1
        result_val[i] = x[i]
        for j in range(1, half_k+1):
            if x[i] < x[i-j]:
                result[i] = 0
                result_val[i] = 0
                break
            if x[i] < x[i+j]:
                result[i] = 0
                result_val[i] = 0
                break
    return result, result_val


### postprocess 1st stage

In [7]:
import glob
import os
import time

import tensorflow as tf
import scipy
import numpy as np
import pandas as pd



def detect_peak(array, valid_mask, kernel_size=5, threshold = 0.05):
    # old (GPU)
    # array = tf.cast(array.reshape(1,-1,1), tf.float32)
    # max_array = tf.nn.max_pool1d(array, ksize=kernel_size, strides=1, padding="SAME")
    # mask = tf.cast(max_array==array, tf.float32)
    # array = (array*mask).numpy().reshape(-1)

    peak_mask, peak_array = detect_peak_kmat(array.astype(np.float64), int(kernel_size))
    pred_mask = peak_mask# * valid_mask
    peak_array = peak_array# * valid_mask
    peak_indices = np.where(peak_array>0)[0]
    peak_val = peak_array[peak_indices]
    mask_over_threshold = peak_val > threshold
    peak_indices = peak_indices[mask_over_threshold]
    peak_val = peak_val[mask_over_threshold]
    peak_results = {"peak_array": peak_array, "peak_indices": peak_indices, "peak_val": peak_val}
    # plt.figure(figsize=(15,4))
    # plt.plot(array[20000:80000])
    # plt.plot(peak_array[20000:80000])
    # plt.show()
    # raise Exception
    return peak_results

def gaussian_smooth(array, sigma):#
    if sigma<=0:
        return array
    array = scipy.ndimage.gaussian_filter1d(array, sigma, mode='reflect')
    return array

def moving_average(array, kernel_size):
    if kernel_size<=0:
        return array
    array = scipy.ndimage.uniform_filter1d(array, kernel_size, mode='reflect')
    return array


def smooth_awake(pred_awake, gauss_factor):
    """
    gaussian smooth
    """
    
    return smooth_pred_awake

def find_event(event_indices, pred_awake, before_length, after_length, awake_threshold=0.5, sleep_threshold=0.5, awake_sleep_diff_threshold=0, lengthlist_for_2ndstage=None, pred_nan=None, pred_scores=None, nans_feat=None):
    """
    take average of state before and after event steps
    """
    before_states = []
    after_states = []
    event_types = []
    for event_index in event_indices:
        before_state = pred_awake[max(0, event_index-before_length):event_index].mean()
        after_state = pred_awake[event_index+1:min(event_index+after_length+1, len(pred_awake))].mean()
        before_states.append(before_state)
        after_states.append(after_state)
        if before_state > after_state + awake_sleep_diff_threshold:
            if before_state > awake_threshold and after_state < sleep_threshold:
                event_type = "onset"
            else:
                event_type = "nan"
        elif after_state > before_state + awake_sleep_diff_threshold:
            if before_state < sleep_threshold and after_state > awake_threshold:
                event_type = "wakeup"
            else:
                event_type = "nan"
        else:
            event_type = "nan"
        event_types.append(event_type)
    event_results = {"event_indices": event_indices, "before_states": before_states, "after_states": after_states, "event_types": event_types}


    if lengthlist_for_2ndstage is not None:
        for length in lengthlist_for_2ndstage:
            event_results[f"before_states_feat_{length}"] = [pred_awake[max(0, event_index-length):event_index].mean() for event_index in event_indices]
            event_results[f"after_states_feat_{length}"] = [pred_awake[event_index+1:min(event_index+length+1, len(pred_awake))].mean() for event_index in event_indices]
            if pred_nan is not None:
                event_results[f"before_nan_feat_{length}"] = [pred_nan[max(0, event_index-length):event_index].mean() for event_index in event_indices]
                event_results[f"after_nan_feat_{length}"] = [pred_nan[event_index+1:min(event_index+length+1, len(pred_awake))].mean() for event_index in event_indices]
            if pred_scores is not None:
                for i, col in enumerate(['pred_switch', "pred_switch10p", "pred_switch10", "pred_switch8", "pred_switch6", "pred_switch4", "pred_switch2"]):
                    # event_results[f"before_{col}_{length//2}"] = [pred_scores[max(0, event_index-length//2):event_index,i].mean() for event_index in event_indices]
                    # event_results[f"after_{col}_{length//2}"] = [pred_scores[event_index+1:min(event_index+length//2+1, len(pred_awake)),i].mean() for event_index in event_indices]
                    event_results[f"befaf_{col}_{length//2}"] = [pred_scores[max(0, event_index-length//2):event_index,i].mean()-pred_scores[event_index+1:min(event_index+length//2+1, len(pred_awake)),i].mean() for event_index in event_indices]
                    # event_results[f"{col}_{length//2}_std"] = [pred_scores[max(0, event_index-length//2):min(event_index+length//2+1, len(pred_awake)),i].std() for event_index in event_indices]
                    # 以下追加1120                    
            # if nans_feat is not None:
            #     for i, col in enumerate(['nan_counter', 'nan_span']):
            #         event_results[f"{col}_{length//2}_mean"] = [nans_feat[max(0, event_index-length//2):min(event_index+length//2+1, len(pred_awake)),i].mean() for event_index in event_indices]
            #         event_results[f"{col}_{length//2}_min"] = [nans_feat[max(0, event_index-length//2):min(event_index+length//2+1, len(pred_awake)),i].min() for event_index in event_indices]
            #         event_results[f"{col}_{length//2}_max"] = [nans_feat[max(0, event_index-length//2):min(event_index+length//2+1, len(pred_awake)),i].max() for event_index in event_indices]

    return event_results

def add_night_group(out_df_single):
    # daily 10000step == 2 pm -> night_no
    out_df_single["offset_step"] = out_df_single['step'] + out_df_single['daily_step'].iloc[0] * Cfg.step_for_a_day - out_df_single['step'].iloc[0]
    out_df_single["night"] = 1 + (out_df_single["offset_step"] + Cfg.step_for_a_day - 10000) // Cfg.step_for_a_day
    return out_df_single

def add_night_features(out_df_single, columns=["pred_nan_counter", "pred_nan_span", "pred_nan"]):
    """
    groupby night features (for 2nd stage)
    """
    out_df_single = add_night_group(out_df_single)
    for col in columns:
        out_df_single[f"{col}_mean"] = out_df_single.groupby("night")[col].transform("mean")
        out_df_single[f"{col}_min"] = out_df_single.groupby("night")[col].transform("min")
        out_df_single[f"{col}_max"] = out_df_single.groupby("night")[col].transform("max")
    
    out_df_single = out_df_single.drop(columns=["offset_step", "night"])
    return out_df_single


def delete_inappropriate_event(out_df_single, prior_conf_dev=0.1, prior_conf_abs=0.5):
    """

    many post process
    -> finally, use second stage model instead of this function


    """
    
    # nightとeventの組み合わせでgroupbyして、最大のscoreをカラムに追加する。差が大きい場合は削除する
    out_df_single["max_score_in_night"] = out_df_single.groupby(["night", "event"])["score"].transform("max")
    out_df_single["score_diff"] = out_df_single["max_score_in_night"] - out_df_single["score"]
    mask_by_diff = out_df_single["score_diff"] < prior_conf_dev
    mask_by_max = (np.logical_and((out_df_single["score"] != out_df_single["max_score_in_night"]),  out_df_single["max_score_in_night"] > prior_conf_abs))==False
    mask = np.logical_and(mask_by_diff, mask_by_max)    
    out_df_single["score"] = out_df_single["score"] * mask + (1-mask) * 0.1 * out_df_single["score"]# 1/10で残す　テンポラリ
    
    # start-endを落とすところは元のやつを使いたい
    # maskが最初にTrueになるindexを取得する
    first_index = np.argmax(mask)
    # maskが最後にTrueになるindexを取得する
    last_index = len(mask) - np.argmax(mask[::-1])
    out_df_single = out_df_single.iloc[first_index:last_index].copy()

    # out_df_single = out_df_single[mask]

 
    if len(out_df_single) == 0:
        return out_df_single

    # onsetの後のonset: wakeupが取れなかったケース。最初のonsetは削除
    valid_mask = np.ones(len(out_df_single))
    # print(len(out_df_single))

    

    current_best_conf = 0.
    current_best_idx = 0
    


    onset_found = False
    wakeup_found = False
    count = 0
    while not onset_found: # 最初のイベントがwake upだった場合は、最初のwake upを削除する
        if out_df_single['event'].iloc[count] == 'onset':
            onset_found = True
        else:
            valid_mask[count] = 0
            count += 1
            if count >= len(out_df_single):
                break

    count = 1
    while not wakeup_found: # 最後のイベントがonsetだった場合は、最後のonsetを削除する
        if out_df_single['event'].iloc[-count] == 'wakeup':
            wakeup_found = True
        else:
            valid_mask[-count] = 0
            count += 1
            if count >= len(out_df_single):
                break

    
    # out_df_single = out_df_single[valid_mask==1]
    updated_score = out_df_single["score"].values * valid_mask + (1-valid_mask) * 0.1 * out_df_single["score"].values# 1/10で残す　テンポラリ
    out_df_single.loc[:, "score"] = updated_score
    # if len(out_df_single) == 0:
    #     return out_df_single

    # too long sleep
    # out_df_single['sleep_length'] = np.repeat(out_df_single['step'].values[1::2] - out_df_single['step'].values[::2], 2)
    # out_df_single = out_df_single[out_df_single['sleep_length'] < 14000]
    # if len(out_df_single) == 0:
    #     return out_df_single

    # 同じnightでonsetもしくはwakeupのどちらかがない場合
    out_df_single["num_onset"] = out_df_single.groupby("night")["event"].transform(lambda x: (x=="onset").sum())
    out_df_single["num_wakeup"] = out_df_single.groupby("night")["event"].transform(lambda x: (x=="wakeup").sum())
    out_df_single["have_sleep"] = np.logical_and((out_df_single["num_onset"]>0), out_df_single["num_wakeup"]>0)
    # out_df_single.loc[:, "score"] = out_df_single.loc[:, "score"] * out_df_single.loc[:, "have_sleep"] + (1-out_df_single.loc[:, "have_sleep"]) * 0.1 * out_df_single.loc[:, "score"]# 1/10で残す　テンポラリ
    



    return out_df_single

def avoid_6_vals(out_df_single, pred_switch, window_size=20):
    """
    6の倍数のところは評価指標上僅かに損をする。(GTが1分単位であり、評価指標が12または6の倍数であるため)
    予測結果のstepが6の倍数の場合は、そこからどちらかにずらすと得をする。
    """
    # out_df_single['step'] = out_df_single['step'] + ((out_df_single['step']%6) == 0).astype(int)
    out_df_single.reset_index(drop=True, inplace=True)
    out_df_single["is_6_multiple"] = (out_df_single['step']%6) == 0
    shifts = np.zeros(len(out_df_single))
    for i in range(len(out_df_single)-1):
        if out_df_single['is_6_multiple'].iloc[i] == True:
            index = out_df_single['peak_indices'].iloc[i]
            before = pred_switch[max(0, index-window_size):index].mean()
            after = pred_switch[index+1:min(index+window_size+1, len(pred_switch))].mean()
            if before > after:
                # out_df_single['step'].iloc[i] -= 1
                # out_df_single.at[i, 'step']. -= 1
                shifts[i] = -1
            else:
                # out_df_single['step'].iloc[i] += 1
                # out_df_single.at[i, 'step']. += 1
                shifts[i] = 1
    out_df_single['step'] = out_df_single['step'] + shifts.astype(int)

    # drop column
    out_df_single = out_df_single.drop(columns=["is_6_multiple"])
    return out_df_single

def avoid_6_vals_v2(out_df_single, pred_switch, window_size=20):
    # 近い分にまるめてしまってから1stepずらす。決め打ちリスキー。

    mins = (out_df_single['step'] % int(Cfg.step_for_15min)) / Cfg.step_for_1min
    important_mins = [0,3,7,11,15]
    minites_near = []
    mins_diff = []
    for m in important_mins:
        mins_diff.append(mins - m)
    mins_diff = np.array(mins_diff)
    mins_diff_abs = np.abs(mins_diff)
    mins_diff_argmin = np.argmin(mins_diff_abs, axis=0)
    mins_near = np.array(important_mins)[mins_diff_argmin]
    # print(mins_near)
    min_mins_diff = mins_diff[mins_diff_argmin, np.arange(len(mins_diff_argmin))]
    new_mins_step = (mins_near * Cfg.step_for_1min + (min_mins_diff > 0) - (min_mins_diff <= 0)).astype(int)
    # print(new_mins_step.shape)
    
    out_df_single['step'] = (out_df_single['step'] // int(Cfg.step_for_15min)) * int(Cfg.step_for_15min) + new_mins_step
    return out_df_single

def predict_averaging(predictions, weights):
    """
    予測値の平均をとる
    peak予測をしたpred_10, pred_8, pred_6, pred_4, pred_2、それぞれの重みを与えて平均をとる
    """
    # pred = np.zeros(len(predictions[0]))
    # for prediction, weight in zip(predictions, weights):
    #     pred += prediction * weight
    # pred /= np.sum(weights)
    pred = (predictions * np.array(weights).reshape(1,-1)).sum(axis=1)# / np.sum(weights)

    return pred


def run_postprocess(df, peak_kernel_size=30, peak_threshold=0.05, awake_threshold=0.5, awake_sleep_diff_threshold=0, sleep_threshold=0.5, event_before_length=30, event_after_length=30, prior_conf_dev=0.1, averaging_weight=[0,1,0.5,0,0,0]):
    """
    pp_params
    - gauss_factor: gaussian smoothのfactor
    - awake_threshold: 0.5
    - sleep_threshold: 0.5
    - peak_kernel_size: 30(min)
    - peak_threshold: 0.05
    - event_before_length: 30(min)
    - event_after_length: 30(min)

    """
    # dataframeは["step", "pred_awake", "pred_switch", "pred_nan"})からなる
    # 複数アウト
    df["pred_switch"] = predict_averaging(df[["pred_switch10p", "pred_switch10", "pred_switch8", "pred_switch6", "pred_switch4", "pred_switch2"]].values, weights=averaging_weight)
    df["pred_switch"] = gaussian_smooth(df["pred_switch"].values, sigma=4)
    nonnan_mask = 1 - df["pred_nan"].values
    peak_results = detect_peak(df['pred_switch'].values, nonnan_mask, kernel_size=int(1+2*peak_kernel_size*Cfg.step_for_1min), threshold=peak_threshold) # 2*は両側
    event_results = find_event(peak_results["peak_indices"], df['pred_awake'].values, int(event_before_length*Cfg.step_for_1min), int(event_after_length*Cfg.step_for_1min), awake_threshold=awake_threshold, sleep_threshold=sleep_threshold, awake_sleep_diff_threshold=awake_sleep_diff_threshold)
    out_df_single = pd.DataFrame({"step": df['step'].values[peak_results["peak_indices"]], "peak_indices": peak_results["peak_indices"], "daily_step": df['daily_step'].values[peak_results["peak_indices"]], "event": event_results['event_types'], "score": peak_results['peak_val']})
    # 一時的にevent=="nan"以外のみ残す
    out_df_single = out_df_single[out_df_single['event']!="nan"]
    if len(out_df_single) > 0:
        out_df_single = add_night_group(out_df_single)
        out_df_single = delete_inappropriate_event(out_df_single, prior_conf_dev)
    if len(out_df_single) > 0:
        out_df_single = avoid_6_vals(out_df_single, df['pred_switch'].values)
    #     # out_df_single = avoid_6_vals_v2(out_df_single, df['pred_switch'].values)
        
    return out_df_single

def make_dataset_for_second_model(df_target, df, peak_kernel_size=30, peak_threshold=0.05, awake_threshold=0.5, awake_sleep_diff_threshold=0, sleep_threshold=0.5, event_before_length=30, event_after_length=30, prior_conf_dev=0.1, averaging_weight=[0,1,0.5,0,0,0]):

    """
    マトリクスを作って予測に対して割り当てられるものを見つける
    夜毎のグループで予測確信度でソートして、何個目までいくとスコアいくつで割り当てられるのか、もしくは全体を見た再スコアリングみたいなものをやってみたい。

    予測点数が多すぎる場合は削るとか、他のやつのスコアも低い場合は押し上げるとか。
    まずはポテンシャルをみたいところ
    """
    lengthlist_for_2ndstage = [12, 24, 60, 120, 240, 360, 720] # sleep awakeの平均値保持用
    night_agg_cols = ["pred_nan_counter", "pred_nan_span", "pred_nan", "pred_switch2", "pred_switch", "pred_awake"]# + [f"sub_feat_{i}" for i in range(11)]
    df["pred_switch"] = predict_averaging(df[["pred_switch10p", "pred_switch10", "pred_switch8", "pred_switch6", "pred_switch4", "pred_switch2"]].values, weights=averaging_weight)
    df["pred_switch"] = gaussian_smooth(df["pred_switch"].values, sigma=4)
    df = add_night_features(df, columns=night_agg_cols)
    nonnan_mask = 1 - df["pred_nan"].values
    peak_results = detect_peak(df['pred_switch'].values, nonnan_mask, kernel_size=int(1+2*peak_kernel_size*Cfg.step_for_1min), threshold=peak_threshold)
    event_results = find_event(peak_results["peak_indices"], df['pred_awake'].values, int(event_before_length*Cfg.step_for_1min), int(event_after_length*Cfg.step_for_1min), awake_threshold=awake_threshold, sleep_threshold=sleep_threshold, 
                                awake_sleep_diff_threshold=awake_sleep_diff_threshold, 
                                lengthlist_for_2ndstage=lengthlist_for_2ndstage,
                                pred_nan=df['pred_nan'].values,
                                pred_scores=df[['pred_switch', "pred_switch10p", "pred_switch10", "pred_switch8", "pred_switch6", "pred_switch4", "pred_switch2"]].values,
                                nans_feat=df[["pred_nan_counter", "pred_nan_span"]].values)
    out_df_single = pd.DataFrame({"step": df['step'].values[peak_results["peak_indices"]], 
            "peak_indices": peak_results["peak_indices"], 
            "daily_step": df['daily_step'].values[peak_results["peak_indices"]], 
            "event": event_results['event_types'], 
            "score": peak_results['peak_val'],
            "score_10p": df['pred_switch10p'].values[peak_results["peak_indices"]],
            "score_10": df['pred_switch10'].values[peak_results["peak_indices"]],
            "score_8": df['pred_switch8'].values[peak_results["peak_indices"]],
            "score_6": df['pred_switch6'].values[peak_results["peak_indices"]],
            "score_4": df['pred_switch4'].values[peak_results["peak_indices"]],
            "score_2": df['pred_switch2'].values[peak_results["peak_indices"]],
            "pred_nan": df['pred_nan'].values[peak_results["peak_indices"]],
            "pred_nan_counter": df['pred_nan_counter'].values[peak_results["peak_indices"]],
            "pred_nan_span": df['pred_nan_span'].values[peak_results["peak_indices"]],
            "pred_awake": df['pred_awake'].values[peak_results["peak_indices"]],
            "before_states": event_results['before_states'],
            "after_states": event_results['after_states'],
            })
    for length in lengthlist_for_2ndstage:
        out_df_single[f"before_states_feat_{length}"] = event_results[f"before_states_feat_{length}"]
        out_df_single[f"after_states_feat_{length}"] = event_results[f"after_states_feat_{length}"]
        out_df_single[f"before_nan_feat_{length}"] = event_results[f"before_nan_feat_{length}"]
        out_df_single[f"after_nan_feat_{length}"] = event_results[f"after_nan_feat_{length}"]
        for i, col in enumerate(['pred_switch', "pred_switch10p", "pred_switch10", "pred_switch8", "pred_switch6", "pred_switch4", "pred_switch2"]):
            # out_df_single[f"before_{col}_{length//2}"] = event_results[f"before_{col}_{length//2}"]
            # out_df_single[f"after_{col}_{length//2}"] = event_results[f"after_{col}_{length//2}"]
            out_df_single[f"befaf_{col}_{length//2}"] = event_results[f"befaf_{col}_{length//2}"]
            # out_df_single[f"{col}_{length//2}_std"] = event_results[f"{col}_{length//2}_std"]
            
        # for i, col in enumerate(['nan_counter', 'nan_span']):
        #     out_df_single[f"{col}_{length//2}_mean"] = event_results[f"{col}_{length//2}_mean"]
        #     out_df_single[f"{col}_{length//2}_min"] = event_results[f"{col}_{length//2}_min"]
        #     out_df_single[f"{col}_{length//2}_max"] = event_results[f"{col}_{length//2}_max"]
    for col in night_agg_cols:
        out_df_single[f"{col}_mean"] = df[f"{col}_mean"].values[peak_results["peak_indices"]]
        out_df_single[f"{col}_min"] = df[f"{col}_min"].values[peak_results["peak_indices"]]
        out_df_single[f"{col}_max"] = df[f"{col}_max"].values[peak_results["peak_indices"]]
    # 一時的にevent=="nan"以外のみ残す
    out_df_single = out_df_single[out_df_single['event']!="nan"]
    if len(out_df_single) > 0:
        out_df_single = add_night_group(out_df_single)

    # shiftも先にのせてしまう。シフト値だけ記憶してもいいけど。
    if len(out_df_single) > 0:
        out_df_single = avoid_6_vals(out_df_single, df['pred_switch'].values)
    
    # add_target
    def diff_step_to_AP(diff_step):
        return np.sum(np.array([diff_step < threshold for threshold in [12, 36, 60, 90, 120, 150, 180, 240, 300, 360]]), axis=0)
    if len(df_target) >0:
        diff_step_matrix = df_target['step'].values.reshape(-1,1) - out_df_single['step'].values.reshape(1,-1)
        diff_step_matrix = np.abs(diff_step_matrix)
        min_diff_step = np.min(diff_step_matrix, axis=0)
        out_df_single['min_diff_step'] = min_diff_step
        out_df_single['best_ap'] = diff_step_to_AP(min_diff_step)/ 10.
    else:
        out_df_single['min_diff_step'] = np.nan
        out_df_single['best_ap'] = 0

    return out_df_single


def make_dataset_for_second_model_ensemble(ensemble_files, train_events, pp_params, save_path="../data/df_second_model.feather"):
    cannot_finds = []
    id_nos = []
    scores = []
    dates = []
    num_events = []
    diff_steps = []
    num_ensemble = len(ensemble_files)
    do_ensemble = True if num_ensemble > 1 else False
    df_second_model = []
    for i, pred_files_abc in enumerate(zip(*ensemble_files)):

        id_no = os.path.basename(pred_files_abc[0]).split("_")[1]
        # sub_data = os.path.join(Cfg.preprocess_dir, "id_" + id_no+"_subfeat.npy")
        df_pred_id = pd.read_parquet(pred_files_abc[0])
        # sub_data = np.load(sub_data)
        # add_cols = [f"sub_feat_{i}" for i in range(sub_data.shape[-1])]
        # df_pred_id[add_cols] = sub_data
        # print(sub_data.shape, df_pred_id.shape)

        # ----- ENSEMBLE
        if do_ensemble:
            for pf in pred_files_abc[1:]:
                df_pred_id_1 = pd.read_parquet(pf)
                cols = [c for c in df_pred_id.columns if "pred_switch" in c] + ["pred_awake"]
                for c in cols:
                    df_pred_id[c] += df_pred_id_1[c]
            for c in cols:
                df_pred_id[c] /= num_ensemble
        # -----

        print(id_no)
        
        solution =  train_events.loc[train_events['series_id']==id_no].copy()
        solution = solution[~np.isnan(solution['step'])]

        out_df_single = make_dataset_for_second_model(solution, df_pred_id, **pp_params)
        out_df_single['series_id'] = id_no
        df_second_model.append(out_df_single)
        
    df_second_model = pd.concat(df_second_model).reset_index(drop=True)
    float32_cols = [col for col in df_second_model.columns if df_second_model[col].dtype=="float64"]
    df_second_model[float32_cols] = df_second_model[float32_cols].astype(np.float32)
    # feather で保存
    df_second_model.to_feather(save_path)
    return df_second_model

def validation_scoring(pred_files, train_events, pp_params, return_df=False):
    out_df = []
    for file in pred_files:
        id_no = os.path.basename(file).split("_")[1]
        df_pred_id = pd.read_parquet(file)
        # df_pred_id["pred_switch"] = df_pred_id["pred_switch"]

        # tmp = train_events[train_events['series_id']==id_no].copy()
        # tmp = tmp[~np.isnan(tmp['step'])]
        # max_step = tmp["step"].max()
        # df_pred_id = df_pred_id[df_pred_id['step'] <= max_step+2000] 終了タイミングわかれば2ポイントあがるのに

        
        out_df_single = run_postprocess(df_pred_id, **pp_params)
        out_df_single['series_id'] = id_no
        out_df.append(out_df_single)

    out_df = pd.concat(out_df).reset_index(drop=True).reset_index().rename(columns={"index": "row_id"})
    
    series_ids = out_df['series_id'].unique()
    print("num series", len(series_ids))
    solution =  train_events.loc[train_events['series_id'].isin(series_ids)].copy()
    solution = solution[~np.isnan(solution['step'])]
    total_score = event_detection_ap(solution, out_df.copy(), tolerances)
    if return_df:
        return total_score, out_df
    else:
        return total_score

def weighted_fusion_ennsemble(df_0, df_1, distance_threshold=12):
    series_ids = df_0['series_id'].unique()
    out_df = []
    for series_id in series_ids:
        df_0_id = df_0[df_0['series_id']==series_id].copy()
        df_1_id = df_1[df_1['series_id']==series_id].copy()
        df_0_id = df_0_id.sort_values("score", ascending=False).reset_index(drop=True)
        df_1_id = df_1_id.sort_values("score", ascending=False).reset_index(drop=True)
        
        steps_0 = df_0_id['step'].values # base
        steps_1 = df_1_id['step'].values
        scores_0 = df_0_id['score'].values # base
        scores_1 = df_1_id['score'].values
        for step, score in zip(steps_1, scores_1):
            dists = np.abs(steps_0 - step)
            argmin = np.argmin(dists)
            min_dist = dists[argmin]
            if min_dist < distance_threshold:
                f_step = steps_0[argmin]
                f_score = scores_0[argmin]
                add_step = step
                add_score = score
                
                new_score = (f_score + add_score)# / 2
                new_step = ((f_step * f_score) + (add_step * add_score)) / (f_score + add_score)
                df_0_id.loc[argmin, "score"] = new_score
                df_0_id.loc[argmin, "step"] = new_step
            else:
                df_0_id = df_0_id.append(df_1_id[df_1_id['step']==step])
            # assignされたもとのものは参照されないようにする
            steps_0[argmin] = 1e10
        out_df.append(df_0_id)
    out_df = pd.concat(out_df).reset_index(drop=True).reset_index() # .rename(columns={"index": "row_id"})
    return out_df


def validation_scoring_ensemble(pred_files, train_events, pp_params, return_df=False):
    num_ensemble = len(pred_files)
    out_df = []
    print("no sort ensemble")
    # pred_files = [sorted(f) for f in pred_files]
    num_files = len(pred_files[0])
    for i in range(num_files):
        id_no = os.path.basename(pred_files[0][i]).split("_")[1]
        
        for j in range(num_ensemble):
            if j==0:
                df_pred_id = pd.read_parquet(pred_files[j][i])
            else:
                _df_pred_id = pd.read_parquet(pred_files[j][i])
                cols = [c for c in _df_pred_id.columns if "pred_switch" in c]
                for c in cols:
                    df_pred_id[c] += _df_pred_id[c]
        
        for c in cols:
            df_pred_id[c] /= num_ensemble
        out_df_single = run_postprocess(df_pred_id, **pp_params)
        out_df_single['series_id'] = id_no
        out_df.append(out_df_single)
    
    out_df = pd.concat(out_df).reset_index(drop=True).reset_index().rename(columns={"index": "row_id"})

    series_ids = out_df['series_id'].unique()
    solution =  train_events.loc[train_events['series_id'].isin(series_ids)].copy()
    solution = solution[~np.isnan(solution['step'])]
    total_score = event_detection_ap(solution, out_df.copy(), tolerances)
    if return_df:
        return total_score, out_df
    else:
        return total_score
    




In [8]:
import numpy as np
import pandas as pd
import pandas.api.types
from typing import Dict, List, Tuple

# tolerances in steps
tolerances = {
    "onset":  [12, 36, 60, 90, 120, 150, 180, 240, 300, 360],
    "wakeup": [12, 36, 60, 90, 120, 150, 180, 240, 300, 360],
}
# 12は(12の倍数)からだと1minしか許容されない。少しずらすだけで2min可能になる
# 90と150は(12の倍数+6)からだとよくないか。ということでピッタリと30secは避ける。(6の倍数秒は避ける)

series_id_column_name = "series_id"
time_column_name = "step"
event_column_name = "event"
score_column_name = "score"
use_scoring_intervals = None

def score(
        solution: pd.DataFrame,
        submission: pd.DataFrame,
        tolerances: Dict[str, List[float]],
        series_id_column_name: str,
        time_column_name: str,
        event_column_name: str,
        score_column_name: str,
        use_scoring_intervals: bool = False,
) -> float:
    
    # Validate metric parameters
    assert len(tolerances) > 0, "Events must have defined tolerances."
    assert set(tolerances.keys()) == set(solution[event_column_name]).difference({'start', 'end'}),\
        (f"Solution column {event_column_name} must contain the same events "
         "as defined in tolerances.")
    assert pd.api.types.is_numeric_dtype(solution[time_column_name]),\
        f"Solution column {time_column_name} must be of numeric type."

    # Validate submission format
    for column_name in [
        series_id_column_name,
        time_column_name,
        event_column_name,
        score_column_name,
    ]:
        if column_name not in submission.columns:
            raise ParticipantVisibleError(f"Submission must have column '{target_name}'.")

    if not pd.api.types.is_numeric_dtype(submission[time_column_name]):
        raise ParticipantVisibleError(
            f"Submission column '{time_column_name}' must be of numeric type."
        )
    if not pd.api.types.is_numeric_dtype(submission[score_column_name]):
        raise ParticipantVisibleError(
            f"Submission column '{score_column_name}' must be of numeric type."
        )

    # Set these globally to avoid passing around a bunch of arguments
    globals()['series_id_column_name'] = series_id_column_name
    globals()['time_column_name'] = time_column_name
    globals()['event_column_name'] = event_column_name
    globals()['score_column_name'] = score_column_name
    globals()['use_scoring_intervals'] = use_scoring_intervals

    return event_detection_ap(solution, submission, tolerances)

def event_detection_ap(
        solution: pd.DataFrame,
        submission: pd.DataFrame,
        tolerances: Dict[str, List[float]],
) -> float:
    # Ensure solution and submission are sorted properly
    solution = solution.sort_values([series_id_column_name, time_column_name])
    submission = submission.sort_values([series_id_column_name, time_column_name])

    # Extract scoring intervals.
    if use_scoring_intervals:
        intervals = (
            solution
            .query("event in ['start', 'end']")
            .assign(interval=lambda x: x.groupby([series_id_column_name, event_column_name]).cumcount())
            .pivot(
                index='interval',
                columns=[series_id_column_name, event_column_name],
                values=time_column_name,
            )
            .stack(series_id_column_name)
            .swaplevel()
            .sort_index()
            .loc[:, ['start', 'end']]
            .apply(lambda x: pd.Interval(*x, closed='both'), axis=1)
        )

    # Extract ground-truth events.
    ground_truths = (
        solution
        .query("event not in ['start', 'end']")
        .reset_index(drop=True)
    )

    # Map each event class to its prevalence (needed for recall calculation)
    class_counts = ground_truths.value_counts(event_column_name).to_dict()

    # Create table for detections with a column indicating a match to a ground-truth event
    detections = submission.assign(matched = False)

    # Remove detections outside of scoring intervals
    if use_scoring_intervals:
        detections_filtered = []
        for (det_group, dets), (int_group, ints) in zip(
            detections.groupby(series_id_column_name), intervals.groupby(series_id_column_name)
        ):
            assert det_group == int_group
            detections_filtered.append(filter_detections(dets, ints))
        detections_filtered = pd.concat(detections_filtered, ignore_index=True)
    else:
        detections_filtered = detections
    # Create table of event-class x tolerance x series_id values
    aggregation_keys = pd.DataFrame(
        [(ev, tol, vid)
         for ev in tolerances.keys()
         for tol in tolerances[ev]
         for vid in ground_truths[series_id_column_name].unique()],
        columns=[event_column_name, 'tolerance', series_id_column_name],
    )

    # Create match evaluation groups: event-class x tolerance x series_id
    detections_grouped = (
        aggregation_keys
        .merge(detections_filtered, on=[event_column_name, series_id_column_name], how='left')
        .groupby([event_column_name, 'tolerance', series_id_column_name])
    )
    ground_truths_grouped = (
        aggregation_keys
        .merge(ground_truths, on=[event_column_name, series_id_column_name], how='left')
        .groupby([event_column_name, 'tolerance', series_id_column_name])
    )
    # Match detections to ground truth events by evaluation group
    detections_matched = []
    for key in aggregation_keys.itertuples(index=False):
        dets = detections_grouped.get_group(key)
        gts = ground_truths_grouped.get_group(key)
        detections_matched.append(
            match_detections(dets['tolerance'].iloc[0], gts, dets)
        )
    detections_matched = pd.concat(detections_matched)
    

    # Compute AP per event x tolerance group
    event_classes = ground_truths[event_column_name].unique()
    ap_table = (
        detections_matched
        .query("event in @event_classes")
        .groupby([event_column_name, 'tolerance']).apply(
            lambda group: average_precision_score(
                group['matched'].to_numpy(),
                group[score_column_name].to_numpy(),
                class_counts[group[event_column_name].iat[0]],
            )
        )
    )
    # Average over tolerances, then over event classes
    mean_ap = ap_table.groupby(event_column_name).mean().sum() / len(event_classes)

    return mean_ap

def _match_detections(
        tolerance: float, ground_truths: pd.DataFrame, detections: pd.DataFrame
) -> pd.DataFrame:
    """Match detections to ground truth events. Arguments are taken from a common event x tolerance x series_id evaluation group."""
    detections_sorted = detections.sort_values(score_column_name, ascending=False).dropna()
    is_matched = np.full_like(detections_sorted[event_column_name], False, dtype=bool)
    gts_matched = set()
    for i, det in enumerate(detections_sorted.itertuples(index=False)):
        best_error = tolerance
        best_gt = None

        for gt in ground_truths.itertuples(index=False):
            error = abs(getattr(det, time_column_name) - getattr(gt, time_column_name))
            if error < best_error and gt not in gts_matched:
                best_gt = gt
                best_error = error

        if best_gt is not None:
            is_matched[i] = True
            gts_matched.add(best_gt)

    detections_sorted['matched'] = is_matched

    return detections_sorted

def match_detections(
        tolerance: float, ground_truths: pd.DataFrame, detections: pd.DataFrame
) -> pd.DataFrame:
    """Match detections to ground truth events. Arguments are taken from a common event x tolerance x series_id evaluation group."""
    detections_sorted = detections.sort_values(score_column_name, ascending=False).dropna()
    is_matched = np.full_like(detections_sorted[event_column_name], False, dtype=bool)
    gts_matched = set()

    det_times = detections_sorted[time_column_name].values.reshape(-1)
    gt_times = ground_truths[time_column_name].values.reshape(-1)
    errors_matrix = np.abs(gt_times[np.newaxis, :] - det_times[:, np.newaxis])
    """
    pred_indices_matrix = np.tile(np.arange(len(det_times))[:,np.newaxis], (1,len(gt_times)))
    gt_values_matrix = np.tile(gt_times[np.newaxis, :], (len(det_times),1))

    cond = errors_matrix < tolerance
    errors = errors_matrix[cond]
    pred_indices = pred_indices_matrix[cond]
    gt_values = gt_values_matrix[cond]
    last_i = -1
    best_gt = None
    best_error = tolerance
    for i, gt, error in zip(pred_indices, gt_values, errors):
        pred_reset = True if i != last_i else False
        if pred_reset and best_gt is not None:
            is_matched[i] = True
            gts_matched.add(best_gt)
        if pred_reset:
            best_gt = None
            best_error = tolerance
        if error < best_error and gt not in gts_matched:
            best_gt = gt
            best_error = error

        last_i = i
    if pred_reset and best_gt is not None:
        is_matched[i] = True
        gts_matched.add(best_gt)
    """

    
    # for i, det_time in enumerate(det_times): # detections_sorted.itertuples(index=False)):
    for i in range(len(det_times)): # detections_sorted.itertuples(index=False)):

        errors = errors_matrix[i] # np.abs(gt_times - det_time)
        # print(errors.shape)
        # print(np.abs(gt_times - det_time).shape)
        # mask = (errors < tolerance) & (~np.isin(gt_indices, list(gts_matched)))
        

        best_error = tolerance
        best_gt = None
        mask = errors < best_error
        errors_masked = errors[mask]
        gt_times_masked = gt_times[mask]

        for error, gt in zip(errors_masked, gt_times_masked):#ground_truths.itertuples(index=False):
            # error = abs(getattr(det, time_column_name) - getattr(gt, time_column_name))
            if  gt not in gts_matched:
                best_gt = gt
                best_error = error

        if best_gt is not None:
            is_matched[i] = True
            gts_matched.add(best_gt)
    

    detections_sorted['matched'] = is_matched

    return detections_sorted


def precision_recall_curve(
        matches: np.ndarray, scores: np.ndarray, p: int
) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    if len(matches) == 0:
        return [1], [0], []

    # Sort matches by decreasing confidence
    idxs = np.argsort(scores, kind='stable')[::-1]
    scores = scores[idxs]
    matches = matches[idxs]

    distinct_value_indices = np.where(np.diff(scores))[0]
    threshold_idxs = np.r_[distinct_value_indices, matches.size - 1]
    thresholds = scores[threshold_idxs]

    # Matches become TPs and non-matches FPs as confidence threshold decreases
    tps = np.cumsum(matches)[threshold_idxs]
    fps = np.cumsum(~matches)[threshold_idxs]

    precision = tps / (tps + fps)
    precision[np.isnan(precision)] = 0
    recall = tps / p  # total number of ground truths might be different than total number of matches

    # Stop when full recall attained and reverse the outputs so recall is non-increasing.
    last_ind = tps.searchsorted(tps[-1])
    sl = slice(last_ind, None, -1)

    # Final precision is 1 and final recall is 0
    return np.r_[precision[sl], 1], np.r_[recall[sl], 0], thresholds[sl]

def average_precision_score(matches: np.ndarray, scores: np.ndarray, p: int) -> float:
    precision, recall, _ = precision_recall_curve(matches, scores, p)
    # Compute step integral
    return -np.sum(np.diff(recall) * np.array(precision)[:-1])





In [9]:
train_events = pd.read_csv(Cfg.train_target_path)


### evaluation

In [32]:

"""

pp_params = {'awake_threshold': 0.01118579240197301, 'sleep_threshold': 0.9741809981418406, 'awake_sleep_diff_threshold': 0.0036734165080717115, 'peak_kernel_size': 4.689964359639259, 'peak_threshold': 0.05135826117540523, 'event_before_length': 68.38372582053368, 'event_after_length': 99.69200455879107, 'prior_conf_dev': 0.29102092096947463}
pp_params["averaging_weight"] = [0.333, 0.333, 0.333, 0.00, 0, 0]

pred_files =  glob.glob("../model/weights/exp04_run_14_SplitStem2fold35epStateSigmoidStratified14000stepDeepSEED111ManyposCh40Lr12_fold0/pred090/*.parquet")# + glob.glob("../model/weights/exp04_run_14_SplitStem2fold35epStateSigmoidStratified14000stepDeepSEED111ManyposCh40Lr12_fold1/pred090/*.parquet") # 0.8142088586829181 (31237, 14)
pred_files =  glob.glob("../model/weights/exp04_run_20_SplitStem2fold35epStateSigmoidStratified18000stepDeepSEED42_fold0/pred090/*.parquet")# + glob.glob("../model/weights/exp04_run_20_SplitStem2fold35epStateSigmoidStratified18000stepDeepSEED42_fold1/pred090/*.parquet")

pred_files =  glob.glob("../model/weights/exp04_run_24_SplitStem2foldSEED111controledStride_fold0/pred090/*.parquet") + glob.glob("../model/weights/exp04_run_24_SplitStem2foldSEED111controledStride_fold1/pred090/*.parquet") # 0.8145448837706926 (31883, 14) 0.814454051763341 (29676, 14)peak 尖らし。
pred_files =  glob.glob("../model/weights/exp04_run_26_SplitStem2foldSEED111controledStride60secdata_fold0/pred090/*.parquet") + glob.glob("../model/weights/exp04_run_26_SplitStem2foldSEED111controledStride60secdata_fold1/pred090/*.parquet") # 0.8131842985751643 (29461, 14)
pred_files =  glob.glob("../model/weights/exp04_run_28_SplitStem2foldSEED42controledStride_fold0/pred090/*.parquet") + glob.glob("../model/weights/exp04_run_28_SplitStem2foldSEED42controledStride_fold1/pred090/*.parquet") # 


# pred_files =  glob.glob("../model/weights/exp03_run_08_s10s8s6s4s2multiouts2fold40epTriModel_fold0/pred/*.parquet")

# pp_params = {'awake_threshold': 0.01118579240197301, 'sleep_threshold': 0.9741809981418406, 'awake_sleep_diff_threshold': 0.0036734165080717115, 'peak_kernel_size': 2, 'peak_threshold': 0.05135826117540523/2, 'event_before_length': 68.38372582053368/2, 'event_after_length': 99.69200455879107/2, 'prior_conf_dev': 0.29102092096947463}
# pred_files = glob.glob("../model/weights/exp02_run_29_s10s8s6s4s2multiouts_25D40epPEncSin4STRIDE2_fold0/pred/*.parquet")# + glob.glob("../model/weights/exp02_run_26_s10s8s6s4s2multiouts_25D40epPEncSin4_fold1/pred/*.parquet") # 0.8055317525988136


# pred_files = []
# for i in range(5):
#    pred_files += glob.glob(f"../model/weights/exp02_run_09_focal_s10s8blend_cut3mix_5FOLD_fold{i}/pred/*.parquet") # 0.7893816681055978 0.7895854315906429 0.7900825314890543, 0.7914190667754986

# pred_files = []
# for i in range(5):
#    pred_files += glob.glob(f"../model/weights/exp02_run_30_s10s8s6s4s2multiouts5fold_fold{i}/pred/*.parquet") # 0.8101882890461795 gauss 0.8113963182104299

# pred_files = []
# for i in range(5):
#    pred_files += glob.glob(f"../model/weights/exp03_run_12_s10s8s6s4s2multiouts2fold35epNormal_fold{i}/pred/*.parquet") # 0.8134289288683368 29719 (0.8144359591850177 , 43014) -> 0.8220663524837813 @ second stage

# pred_files = []
# for i in range(5):
#    pred_files += glob.glob(f"../model/weights/exp04_run_08_SplitStem2fold35epStateFocal25pStratified_fold{i}/pred/*.parquet") # 0.813979615059301 31202 (0.8147061899749469, 46319)




# pred_files = []
# for i in range(5):
#    pred_files += glob.glob(f"../model/weights/exp04_run_10_SplitStem2fold35epStateFocal25pStratified14000stepDeepSEED111_fold{i}/pred090/*.parquet") # 0.8190526163294717 
# print(len(pred_files))
# pred_A = pred_files

# pred_files = []
# for i in range(5):
#    pred_files += glob.glob(f"../model/weights/exp04_run_11_SplitStem2fold35epStateFocal25pStratified14000stepDeepSEED42_fold{i}/pred/*.parquet") # 0.8167204674976697 (31879, 14)

# pred_B = pred_files

# pred_files = []
# for i in range(5):
#    pred_files += glob.glob(f"../model/weights/exp04_run_15_SplitStem2fold35epStateSigmoidStratified14000stepDeepSEED111ManyposCh40Lr12_fold{i}/pred090/*.parquet") # 0.8210976525766087 (20031, 14) -> 0.8185172733765049 (19338, 14)

pred_files = []
for i in range(1,2):
    # pred_files += glob.glob(f"../model/weights/exp04_run_16_SplitStem2fold35epStateSigmoidStratified14000stepDeepSEED42ManyposCh40Lr12_fold{i}/pred090/*.parquet") # 0.8186020290947744 (30023, 14)
    # pred_files += glob.glob(f"../model/weights/exp04_run_15_SplitStem2fold35epStateSigmoidStratified14000stepDeepSEED111ManyposCh40Lr12_fold{i}/pred090/*.parquet") # 0.8202121238443345 (30185, 14)
    # pred_files += glob.glob(f"../model/weights/exp04_run_29_SplitStem5foldSEED111controledStride_fold{i}/pred090/*.parquet")# seed42 0.8202335795874665 (30831, 14), seed 111 0.8202097089235567 (30409, 14)
    # pred_files += glob.glob(f"../model/weights/exp04_run_30_SplitStem2foldSEED111shift_target_fold{i}/pred090/*.parquet")# seed42 0.8169063870836835 (30998, 14), seed111 0.8149626248679888 (30780, 14)

    # pred_files += glob.glob(f"../model/weights/exp04_run_36_SplitStem5foldSEED0controledStridev2_fold{i}/pred090/*.parquet") # 42: 0.8212767417753822 (30821, 14) 111:0.8231633388171156 (30972, 14) 0:0.8226272950876579 (31140, 14)
    pred_files += glob.glob(f"../model/weights/exp04_run_38_SplitStem2foldSEED42normal_fold{i}/pred090/*.parquet") # 


# pp_params["averaging_weight"] = [0.333+0.1, 0.333-0.04, 0.333-0.04, 0.00, 0, 0] # 0.8211263988238442 (29130, 14)

# pp_params["averaging_weight"] = [0.33, 0.33, 0.33, 0.00, 0, 0] # 0.8213470468009865 (30300, 14) 0.823193701744753 (30422, 14)




# pp_params["averaging_weight"] = [0.1, 0.3, 0.4, 0.2, 0.0, 0] # 0.8170802292733318 (31964, 14) 0.8168524707260241 (30507, 14) 0.8169063870836835 (30998, 14)
# pp_params['peak_threshold'] += 0.012

# pp_params["averaging_weight"] = [0.333, 0.333, 0.333, 0.00, 0, 0] # 0.8153428343652527 (30980, 14)

# pp_params["averaging_weight"] = [0.133, 0.333, 0.333, 0.20, 0, 0] # 0.8165805440854592 (37663, 14) + 0.005 0.8163801737141492 (33827, 14) 0.8163981574272584 (31083, 14)


print(len(pred_files))


# pred_A_base = [os.path.basename(f) for f in pred_A]
# pred_B_base = [os.path.basename(f) for f in pred_B]
# pred_A = [pred_A[i] for i in np.argsort(pred_A_base)]
# pred_B = [pred_B[i] for i in np.argsort(pred_B_base)]


# pp_params = {'awake_threshold': 0.6504054224129343, 'sleep_threshold': 0.9567267743274901, 'peak_kernel_size': 48.42466176636616, 'peak_threshold': 0.08966659103875652, 'event_before_length': 38.308094765905594, 'event_after_length': 57.22569409093988, 'prior_conf_dev': 0.19796846319117362}

debugging = False
if debugging:
    ref = ['038441c925bb', '03d92c9f6f8a', '0402a003dae9', '04f547b8017d',
        '05e1944c3818', '062cae666e2a', '062dbd4c95e6', '08db4255286f',
        '0a96f4993bd7', '0cd1e3d0ed95', '0ce74d6d2106', '0cfc06c129cc',
        '0d0ad1e77851', '0dee4fda51c3', '0ec9fc461819', '0ef7d94fde99',
        '0f572d690310', '0f9e60a8e56d', '10469f6765bf', '1087d7b0ff2e']
    # ref = ['038441c925bb', '03d92c9f6f8a', '0402a003dae9', '04f547b8017d',
    #    '05e1944c3818', '062cae666e2a', '062dbd4c95e6', '08db4255286f',
    #    '0a96f4993bd7', '0cd1e3d0ed95', '0ce74d6d2106', '0cfc06c129cc',
    #    '0d0ad1e77851', '0dee4fda51c3', '0ec9fc461819', '0ef7d94fde99',
    #    '0f572d690310', '0f9e60a8e56d', '10469f6765bf', '1087d7b0ff2e',
    #    '10f8bc1f7b07', '12d01911d509', '1319a1935f48', '137771d19ca2',
    #    '137b99e936ab', '13b4d6a01d27', '148471991ffb', '154fe824ed87',
    #    '16fe2798ed0f', '1716cd4163b2', '1762ab70ec76', '188d4b7cd28b',
    #    '18a0ca03431d', '18b61dd5aae8', '1955d568d987', '1b92be89db4c',
    #    '1c7c0bad1263', '1d4569cbac0f', '1e6717d93c1d', '1f96b9668bdf',
    #    '207eded97727', '25e2b3dd9c3b', '2654a87be968', '27f09a6a858f',
    #    '280e08693c6d', '292a75c0b94e', '29c75c018220', '29d3469bd15d',
    #    '2b0a1fa8eba8', '2b8d87addea9', '2cd2340ca14d', '2e9ced2c7976',
    #    '2f7504d0f426', '2fbbee1a38e3', '2fc653ca75c7', '31011ade7c0a',
    #    '3318a0e3ed6f', '33ceeba8918a', '3452b878e596', '349c5562ee2c',
    #    '35826366dfc7', '361366da569e', '3664fe9233f9', '3665c86afaf5',
    #    '390b487231ce', '3a9a9dc2cbd9', '3aceb17ef7bd', '3be1545083b7',
    #    '3be2f86c3e45', '3c336d6ba566', '3d53bfea61d6', '3df0da2e5966',
    #    '405df1b41f9f', '40dce6018935', '416354edd92a', '449766346eb1',
    #    '44a41bba1ee7', '44d8c02b369e', '4743bdde25df', '483d6545417f',
    #    '4a31811f3558', '4ab54be1a403', '4ac356361be9', '4b45c36f8f5a',
    #    '4feda0596965', '519ae2d858b0', '51b23d177971', '51c49c540b4e',
    #    '51fdcc8d9fe7', '559ffb7c166a', '55a47ff9dc8a', '55b7f5c99930',
    #    '599ca4ed791b', '5aad18e7ce64', '5acc9d63b5fd', '5c088d7e916c',
    #    '5c55a5e717d6', '5e816f11f5c3', '5f40907ec171', '5f76965e10cf']
    # pred_filesの中から、refに含まれるものだけを抽出する
    pred_files = [file for file in pred_files if os.path.basename(file).split("_")[1] in ref]
    ref_intersection = [os.path.basename(pred_file).split("_")[1] for pred_file in pred_files]
    print(ref_intersection)



score, _df = validation_scoring(pred_files, train_events.copy(), pp_params, return_df=True)
print(score, _df.shape)


# # SIMPLE ENSEMBLE
# l_pred_files = [pred_A, pred_B]
# score, _df = validation_scoring_ensemble(l_pred_files, train_events.copy(), pp_params, return_df=True) # 0.7593352370275017
# print(score, _df.shape)



# WBF Ensemble
# test_length = 300
# score_A, pred_A_df = validation_scoring(pred_A[:test_length], train_events.copy(), pp_params, return_df=True)
# score_B, pred_B_df = validation_scoring(pred_B[:test_length], train_events.copy(), pp_params, return_df=True)
# score_AB_av = validation_scoring_ensemble([pred_A[:test_length], pred_B[:test_length]], train_events.copy(), pp_params)
# pred_AB_wbf_df = weighted_fusion_ennsemble(pred_A_df, pred_B_df, distance_threshold=120)

# series_ids = pred_AB_wbf_df['series_id'].unique()
# solution =  train_events.loc[train_events['series_id'].isin(series_ids)].copy()
# solution = solution[~np.isnan(solution['step'])]
# score_AB_wbf = event_detection_ap(solution, pred_AB_wbf_df.copy(), tolerances)

# 0.8185768489903773 (20606, 14)0.8202611423370336 (17670, 14)


# 0.8176806963633741 (17312, 14)

"""

138
num series 138
0.8108576871662161 (13373, 14)


### make dataset for 2nd

In [17]:
pp_params = {'awake_threshold': 0.01118579240197301, 'sleep_threshold': 0.9741809981418406, 'awake_sleep_diff_threshold': 0.0036734165080717115, 
            'peak_kernel_size': 4.689964359639259, 'peak_threshold': 0.05135826117540523, 
            'event_before_length': 68.38372582053368, 'event_after_length': 99.69200455879107, 'prior_conf_dev': 0.29102092096947463,
            "averaging_weight": [0.333, 0.333, 0.333, 0.00, 0, 0]}

num_folds = 2 if Cfg.IS_DEBUG=="True" else 5
pred_files_A0 = []
for i in range(5):
    pred_files_A0 += glob.glob(os.path.join(Cfg.weight_dir_1dcnn, f"exp00_run_00_SplitStem{num_folds}foldSEED42controledStride_fold{i}/pred090/*.parquet"))

pred_files_A1 = []
for i in range(5):
    pred_files_A1 += glob.glob(os.path.join(Cfg.weight_dir_1dcnn, f"exp00_run_00_SplitStem{num_folds}foldSEED111controledStride_fold{i}/pred090/*.parquet")) # 

pred_files_B0 = []
for i in range(5):
    pred_files_B0 += glob.glob(os.path.join(Cfg.weight_dir_1dcnn, f"exp00_run_01_SplitStem{num_folds}foldSEED42normal_fold{i}/pred090/*.parquet"))

pred_files_B1 = []
for i in range(5):
    pred_files_B1 += glob.glob(os.path.join(Cfg.weight_dir_1dcnn, f"exp00_run_01_SplitStem{num_folds}foldSEED111normal_fold{i}/pred090/*.parquet"))

# sort
pred_A0_base = [os.path.basename(f) for f in pred_files_A0]
pred_A1_base = [os.path.basename(f) for f in pred_files_A1]
pred_B0_base = [os.path.basename(f) for f in pred_files_B0]
pred_B1_base = [os.path.basename(f) for f in pred_files_B1]

pred_files_A0 = [pred_files_A0[i] for i in np.argsort(pred_A0_base)]
pred_files_A1 = [pred_files_A1[i] for i in np.argsort(pred_A1_base)]
pred_files_B0 = [pred_files_B0[i] for i in np.argsort(pred_B0_base)]
pred_files_B1 = [pred_files_B1[i] for i in np.argsort(pred_B1_base)]

print("---make dataset for second model---")

with timer("ensemble dataset"):
    ensemble_files =  [pred_files_A0, pred_files_A1]
    make_dataset_for_second_model_ensemble(ensemble_files, train_events, pp_params, save_path=os.path.join(Cfg.preprocess_dir, "df_second_model_0.feather"))

with timer("ensemble dataset"):
    ensemble_files =  [pred_files_B0, pred_files_B1]
    make_dataset_for_second_model_ensemble(ensemble_files, train_events, pp_params, save_path=os.path.join(Cfg.preprocess_dir, "df_second_model_1.feather"))



[MEMUSE] memory usage (in before ensemble dataset): 16392.45MB (25.1%)
[ensemble dataset] 0.000sec
[MEMUSE] memory usage (in after ensemble dataset): 16392.45MB (25.1%)
[MEMUSE] memory usage (in before ensemble dataset): 16392.45MB (25.1%)
[ensemble dataset] 0.000sec
[MEMUSE] memory usage (in after ensemble dataset): 16392.45MB (25.1%)
[MEMUSE] memory usage (in before ensemble dataset): 16392.45MB (25.1%)
[ensemble dataset] 0.000sec
[MEMUSE] memory usage (in after ensemble dataset): 16392.45MB (25.1%)
[MEMUSE] memory usage (in before ensemble dataset): 16392.45MB (25.1%)
038441c925bb
03d92c9f6f8a
0402a003dae9
04f547b8017d
05e1944c3818
062cae666e2a
062dbd4c95e6
08db4255286f
0a96f4993bd7
0cd1e3d0ed95
0ce74d6d2106
0cfc06c129cc
0d0ad1e77851
0dee4fda51c3
0ec9fc461819
0ef7d94fde99
0f572d690310
0f9e60a8e56d
10469f6765bf
1087d7b0ff2e
10f8bc1f7b07
12d01911d509
1319a1935f48
137771d19ca2
137b99e936ab
13b4d6a01d27
148471991ffb
154fe824ed87
16fe2798ed0f
1716cd4163b2
1762ab70ec76
188d4b7cd28b
18a0ca